# Implementing comment verification/spam detection

### imports and utility functions definition

In [1]:
import pandas as pd
from collections import defaultdict
from PersianStemmer import PersianStemmer
import re

In [2]:
def remove_punctuation(text):
    text=str(text)
    import string
    punct = string.punctuation + '.،!"#$%&|×/:؛,][\}{«»<>؟'

    return text.translate({ord(p): " " for p in punct})

def stopwords(text):
    text = [word for word in text.split() if word not in sw]
    return " ".join(text)

def remove_enter (comment):
    comment =re.sub(r'[(\r\n)+]|[\r\n]+', ' ', comment)
    comment =re.sub(r'\s+', ' ', comment)
    return comment

def stemmingText (arr):
    for i,val in enumerate(arr):
        arr[i] = ps.run(val)
    return (" ".join(arr))

#### functions for calculating probabilities consist of simple probability and conditional

In [3]:
def cond_prob (word, label):
    if label==0:
        return (not_spam_dic[word]+1)/(word_count+notspam_word_count)
    elif label==1:
        return (spam_dic[word]+1)/(word_count+spam_word_count)
    else:
        return 0

def prob(label, comment):
    words = str(comment).split()
    class_prob = spam_prob if label == 1 else NotSpam_prob
#   a postrior probability (text|label)
    result = class_prob
    for word in words:
        if (cond_probs[word]):
            result *= cond_probs[word][label]
    return result

#### Main Implementation

In [4]:
fileAddress='./resources/stopwords-fa.txt'
with open(fileAddress,encoding='utf-8') as file:
    sw=file.read().splitlines()
del sw[0]

In [5]:
ps = PersianStemmer()

In [6]:
data= pd.read_csv('./train.csv')
test= pd.read_csv('./test.csv')

In [7]:
data.fillna(' ',inplace=True)
data['comment'] = data['title'] +" "+ data['comment']

In [8]:
# data['comment'] = data['comment'].apply(remove_enter)
data['comment'] = data['comment'].apply(remove_punctuation)
data['comment'] = data['comment'].apply(stopwords)

In [9]:
for i in range(data.shape[0]):
    data.at[i, 'comment'] = stemmingText(str(data.at[i,'comment']).split())

In [10]:
data=data[['comment','verification_status']]
data['tokens'] = data['comment'].str.split()

In [11]:
words=defaultdict(int)
for comment in data['comment'].values:
    for elem in comment.split(' '):
            words[elem]+=1

In [12]:
word_count = len(words)

spam_prob = data[data['verification_status']==1].shape[0]/data.shape[0]
NotSpam_prob = 1-spam_prob

In [13]:
spam_df = data[data['verification_status']==1]
nspam_df = data[data['verification_status']==0]

In [14]:
spam_dic=defaultdict(int)
for comment in spam_df['comment'].values:
    for elem in comment.split(' '):
            spam_dic[elem]+=1

not_spam_dic=defaultdict(int)
for comment in nspam_df['comment'].values:
    for elem in comment.split(' '):
            not_spam_dic[elem]+=1

In [15]:
#number of total words in not spam with repeats
notspam_word_count=0
for key in not_spam_dic.keys():
    notspam_word_count += not_spam_dic[key]

#number of total words in not spam with repeats
spam_word_count=0
for key in spam_dic.keys():
    spam_word_count += spam_dic[key]

In [16]:
#conditional probabilities
cond_probs=defaultdict(int)
for word in words.keys():
    cond_probs[word] = (cond_prob(word,0), cond_prob(word,1))

# test

In [17]:
test['text']= test['title']+ ' ' + test['comment']
test['text'] = test['text'].apply(remove_punctuation)
for i in range(test.shape[0]):
    test.at[i, 'comment'] = stemmingText(str(test.at[i,'comment']).split())

verification_status=[]
for i in range(test.shape[0]):
    if str(test.loc[i].title)==str(test.loc[i].comment):
        verification_status.append(1)
    else:
        if prob(0, test.loc[i].text) < prob(1, test.loc[i].text):
            verification_status.append(1)
        else:
            verification_status.append(0)
df = pd.DataFrame(list(zip(test['id'].values, verification_status)), columns =['id', 'verification_status'])
df.to_csv(r'./ans.csv',index=False)